In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')


In [2]:
heart_df=pd.read_csv("resources/cleaner_bmi.csv")
heart_df.head()

,id,age (years),gender,BMI,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,50,2,22.015308,110,80,1,1,0,0,1,0
1,1,55,1,34.850994,140,90,3,1,0,0,1,1
2,2,51,1,23.489511,130,70,3,1,0,0,0,1
3,3,48,2,28.742724,150,100,1,1,0,0,1,1
4,4,47,1,22.923381,100,60,1,1,0,0,0,0


In [3]:
y = heart_df["cardio"]
X = heart_df.drop(["id","cardio"], axis=1)

In [4]:
X.describe()

,age (years),gender,BMI,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
count,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000
mean,52.820562,1.350385,27.021492,126.259011,81.153845,1.356141,1.219171,0.087592,0.052507,0.804523
std,6.768891,0.477094,4.508784,16.026882,9.121909,0.672405,0.564869,0.282703,0.223049,0.396570
min,29.000000,1.000000,14.495481,90.000000,60.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,48.000000,1.000000,23.822696,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000
50%,53.000000,1.000000,26.193395,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000
75%,58.000000,2.000000,29.728700,140.000000,90.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,64.000000,2.000000,48.105155,197.000000,126.000000,3.000000,3.000000,1.000000,1.000000,1.000000


In [27]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)
X_train.shape

(49379, 10)

In [28]:
heart_df["cardio"].value_counts()

0    33661
1    32178
Name: cardio, dtype: int64

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10


# Create a Sequential instatnce
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 50)                550       
                                                                 
 dense_9 (Dense)             (None, 20)                1020      
                                                                 
 dense_10 (Dense)            (None, 10)                210       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,791
Trainable params: 1,791
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq='epoch')

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
1529/1544 [============================>.] - ETA: 0s - loss: 0.5572 - accuracy: 0.7220
Epoch 1: saving model to checkpoints\weights.01.hdf5
1544/1544 [==============================] - 3s 2ms/step - loss: 0.5569 - accuracy: 0.7224
Epoch 2/50
1522/1544 [============================>.] - ETA: 0s - loss: 0.5481 - accuracy: 0.7308
Epoch 2: saving model to checkpoints\weights.02.hdf5
1544/1544 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7305
Epoch 3/50
1513/1544 [============================>.] - ETA: 0s - loss: 0.5464 - accuracy: 0.7318
Epoch 3: saving model to checkpoints\weights.03.hdf5
1544/1544 [==============================] - 3s 2ms/step - loss: 0.5467 - accuracy: 0.7315
Epoch 4/50
1515/1544 [============================>.] - ETA: 0s - loss: 0.5456 - accuracy: 0.7329
Epoch 4: saving model to checkpoints\weights.04.hdf5
1544/1544 [==============================] - 3s 2ms/step - loss: 0.5458 - accuracy: 0.7327
Epoch 5/50
1523/1544 [==========

Epoch 35/50
1537/1544 [============================>.] - ETA: 0s - loss: 0.5391 - accuracy: 0.7364
Epoch 35: saving model to checkpoints\weights.35.hdf5
1544/1544 [==============================] - 3s 2ms/step - loss: 0.5392 - accuracy: 0.7363
Epoch 36/50
1521/1544 [============================>.] - ETA: 0s - loss: 0.5382 - accuracy: 0.7378
Epoch 36: saving model to checkpoints\weights.36.hdf5
1544/1544 [==============================] - 3s 2ms/step - loss: 0.5385 - accuracy: 0.7374
Epoch 37/50
1526/1544 [============================>.] - ETA: 0s - loss: 0.5391 - accuracy: 0.7375
Epoch 37: saving model to checkpoints\weights.37.hdf5
1544/1544 [==============================] - 3s 2ms/step - loss: 0.5389 - accuracy: 0.7377
Epoch 38/50
1531/1544 [============================>.] - ETA: 0s - loss: 0.5382 - accuracy: 0.7368
Epoch 38: saving model to checkpoints\weights.38.hdf5
1544/1544 [==============================] - 3s 2ms/step - loss: 0.5385 - accuracy: 0.7366
Epoch 39/50
1543/1544 [=

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {round(model_loss*100)}, Accuracy: {round(model_accuracy*100,2)}")

515/515 - 1s - loss: 0.5401 - accuracy: 0.7361 - 705ms/epoch - 1ms/step
Loss: 54, Accuracy: 73.61


In [36]:
nn.save("heart_nn.h5")

In [37]:
from tensorflow.keras.models import load_model
import json

# Load the model from the .h5 file
heart = load_model('heart_nn.h5')

# Convert the model to a JSON string
heart_nn_json = heart.to_json()

# Save the JSON string to a file
with open('heart_nn.json', 'w') as json_file:
    json_file.write(heart_nn_json)